In [ ]:
"""
This script can be used to evaluate a trained model on 3D pose/shape and masks/part segmentation. You first need to download the datasets and preprocess them.
Example usage:
```
python3 eval.py --checkpoint=data/model_checkpoint.pt --dataset=h36m-p1 --log_freq=20
```
Running the above command will compute the MPJPE and Reconstruction Error on the Human3.6M dataset (Protocol I). The ```--dataset``` option can take different values based on the type of evaluation you want to perform:
1. Human3.6M Protocol 1 ```--dataset=h36m-p1```
2. Human3.6M Protocol 2 ```--dataset=h36m-p2```
3. 3DPW ```--dataset=3dpw```
4. LSP ```--dataset=lsp```
5. MPI-INF-3DHP ```--dataset=mpi-inf-3dhp```
"""

import torch
from torch.utils.data import DataLoader
import numpy as np
import cv2
import os
import argparse
import json
from collections import namedtuple
from tqdm import tqdm
import torchgeometry as tgm

import config
import constants
from models import hmr, SMPL
from datasets import BaseDataset
from utils.imutils import uncrop
from utils.pose_utils import reconstruction_error
from utils.part_utils import PartRenderer

from scipy.spatial.transform import Rotation as R

# Define command-line arguments
parser = argparse.ArgumentParser()
parser.add_argument('--checkpoint', default=None, help='Path to network checkpoint')
parser.add_argument('--dataset', default='h36m-p1', choices=['h36m-p1', 'h36m-p2', 'lsp', '3dpw', 'mpi-inf-3dhp'], help='Choose evaluation dataset')
parser.add_argument('--log_freq', default=50, type=int, help='Frequency of printing intermediate results')
parser.add_argument('--batch_size', default=32, help='Batch size for testing')
parser.add_argument('--shuffle', default=False, action='store_true', help='Shuffle data')
parser.add_argument('--num_workers', default=8, type=int, help='Number of processes for data loading')
parser.add_argument('--result_file', default=None, help='If set, save detections to a .npz file')

def run_evaluation(model, dataset_name, dataset, result_file,
                   batch_size=32, img_res=224, 
                   num_workers=32, shuffle=False, log_freq=50):
    """Run evaluation on the datasets and metrics we report in the paper. """

    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # Transfer model to the GPU
    model.to(device)

    # Load SMPL model
    smpl_neutral = SMPL(config.SMPL_MODEL_DIR,
                        create_transl=False).to(device)
    smpl_male = SMPL(config.SMPL_MODEL_DIR,
                     gender='male',
                     create_transl=False).to(device)
    smpl_female = SMPL(config.SMPL_MODEL_DIR,
                       gender='female',
                       create_transl=False).to(device)
    
    renderer = PartRenderer()
    
    # Regressor for H36m joints
    J_regressor = torch.from_numpy(np.load(config.JOINT_REGRESSOR_H36M)).float()
    
    save_results = result_file is not None
    # Disable shuffling if you want to save the results
    if save_results:
        shuffle=False
    # Create dataloader for the dataset
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
    
    # Pose metrics
    # MPJPE and Reconstruction error for the non-parametric and parametric shapes
    mpjpe = np.zeros(len(dataset))
    recon_err = np.zeros(len(dataset))
    mpjpe_smpl = np.zeros(len(dataset))
    recon_err_smpl = np.zeros(len(dataset))
    
    # Including mean orientation error
    moe = np.zeros(len(dataset))

    # Shape metrics
    # Mean per-vertex error
    shape_err = np.zeros(len(dataset))
    shape_err_smpl = np.zeros(len(dataset))

    # Mask and part metrics
    # Accuracy
    accuracy = 0.
    parts_accuracy = 0.
    # True positive, false positive and false negative
    tp = np.zeros((2,1))
    fp = np.zeros((2,1))
    fn = np.zeros((2,1))
    parts_tp = np.zeros((7,1))
    parts_fp = np.zeros((7,1))
    parts_fn = np.zeros((7,1))
    # Pixel count accumulators
    pixel_count = 0
    parts_pixel_count = 0

    # Store SMPL parameters
    smpl_pose = np.zeros((len(dataset), 72))
    smpl_betas = np.zeros((len(dataset), 10))
    smpl_camera = np.zeros((len(dataset), 3))
    pred_joints = np.zeros((len(dataset), 17, 3))

    eval_pose = False
    eval_masks = False
    eval_parts = False
    eval_orientation = False # Adding the orientation parameter
    # Choose appropriate evaluation for each dataset
    if dataset_name == 'h36m-p1' or dataset_name == 'h36m-p2' or dataset_name == 'mpi-inf-3dhp':
        eval_pose = True
    elif dataset_name == 'lsp':
        eval_masks = True
        eval_parts = True
        annot_path = config.DATASET_FOLDERS['upi-s1h']
    elif dataset_name == '3dpw':
        eval_orientation = True
        eval_pose = True
        

    joint_mapper_h36m = constants.H36M_TO_J17 if dataset_name == 'mpi-inf-3dhp' else constants.H36M_TO_J14
    joint_mapper_gt = constants.J24_TO_J17 if dataset_name == 'mpi-inf-3dhp' else constants.J24_TO_J14
    # Iterate over the entire dataset
    for step, batch in enumerate(tqdm(data_loader, desc='Eval', total=len(data_loader))):
        # Get ground truth annotations from the batch
        gt_pose = batch['pose'].to(device)
        gt_betas = batch['betas'].to(device)
        gt_vertices = smpl_neutral(betas=gt_betas, body_pose=gt_pose[:, 3:], global_orient=gt_pose[:, :3]).vertices
        images = batch['img'].to(device)
        gender = batch['gender'].to(device)
        curr_batch_size = images.shape[0]
        
        with torch.no_grad():
            pred_rotmat, pred_betas, pred_camera = model(images)
            pred_output = smpl_neutral(betas=pred_betas, body_pose=pred_rotmat[:,1:], global_orient=pred_rotmat[:,0].unsqueeze(1), pose2rot=False)
            pred_vertices = pred_output.vertices

        if save_results:
            rot_pad = torch.tensor([0,0,1], dtype=torch.float32, device=device).view(1,3,1)
            rotmat = torch.cat((pred_rotmat.view(-1, 3, 3), rot_pad.expand(curr_batch_size * 24, -1, -1)), dim=-1)
            pred_pose = tgm.rotation_matrix_to_angle_axis(rotmat).contiguous().view(-1, 72)
            smpl_pose[step * batch_size:step * batch_size + curr_batch_size, :] = pred_pose.cpu().numpy()
            smpl_betas[step * batch_size:step * batch_size + curr_batch_size, :]  = pred_betas.cpu().numpy()
            smpl_camera[step * batch_size:step * batch_size + curr_batch_size, :]  = pred_camera.cpu().numpy()
        
        # Orientation evaluation
        if eval_orientation:
            # Get ground truth orientation (already stored in gt_pose)
            gt_rotvec = torch.zeros(curr_batch_size,24,3) # Have to have an array of this shape to input into the rotation object (from 32,72 to 32,24,3)
            i = 0
            for row in gt_pose:
                gt_rotvec[i] = torch.reshape(row,(24, -1))
                i+=1

            gt_euler = np.zeros((curr_batch_size, 24, 3)) # Using numpy here because it works with the rotation library
            i = 0
            for row in gt_rotvec:
                r = R.from_rotvec(row)
                gt_euler[i] = R.as_euler(r, 'xyz', degrees=True)
                i+=1
            
            
            # Get Euler representation of the predictions too:
            
            pred_euler = np.zeros((curr_batch_size,24,3)) # Has to be a numpy array because it works with Rotation

            # For each row in pred_rotmat convert it to a Rotation object and write it into a corresponding
            # row in pred_euler as Euler angles

            i=0
            for row in pred_rotmat:
                r = R.from_dcm(row.cpu())
                pred_euler[i] = R.as_euler(r, 'xyz', degrees=True)
                i+=1
                
            orientation_error = torch.sqrt((torch.from_numpy(gt_euler[:,1:,:]).to(device) -
                                            torch.from_numpy(pred_euler[:,1:,:]).to(device))**2).sum(dim=-1).mean(dim=-1)
            print("orientation_error")
            print(orientation_error)
            print()
            
            moe[step * batch_size:step * batch_size + curr_batch_size] = orientation_error.cpu()
            
        # 3D pose evaluation
        if eval_pose:
            # Regressor broadcasting
            J_regressor_batch = J_regressor[None, :].expand(pred_vertices.shape[0], -1, -1).to(device)
            # Get 14 ground truth joints
            if 'h36m' in dataset_name or 'mpi-inf' in dataset_name:
                gt_keypoints_3d = batch['pose_3d'].cuda()
                gt_keypoints_3d = gt_keypoints_3d[:, joint_mapper_gt, :-1]
            # For 3DPW get the 14 common joints from the rendered shape
            else:
                gt_vertices = smpl_male(global_orient=gt_pose[:,:3], body_pose=gt_pose[:,3:], betas=gt_betas).vertices 
                gt_vertices_female = smpl_female(global_orient=gt_pose[:,:3], body_pose=gt_pose[:,3:], betas=gt_betas).vertices 
                gt_vertices[gender==1, :, :] = gt_vertices_female[gender==1, :, :]
                gt_keypoints_3d = torch.matmul(J_regressor_batch, gt_vertices)
                gt_pelvis = gt_keypoints_3d[:, [0],:].clone()
                gt_keypoints_3d = gt_keypoints_3d[:, joint_mapper_h36m, :]
                gt_keypoints_3d = gt_keypoints_3d - gt_pelvis 


            # Get 14 predicted joints from the mesh
            pred_keypoints_3d = torch.matmul(J_regressor_batch, pred_vertices)
            if save_results:
                pred_joints[step * batch_size:step * batch_size + curr_batch_size, :, :]  = pred_keypoints_3d.cpu().numpy()
            pred_pelvis = pred_keypoints_3d[:, [0],:].clone()
            pred_keypoints_3d = pred_keypoints_3d[:, joint_mapper_h36m, :]
            pred_keypoints_3d = pred_keypoints_3d - pred_pelvis 

            # Absolute error (MPJPE)
            error = torch.sqrt(((pred_keypoints_3d - gt_keypoints_3d) ** 2).sum(dim=-1)).mean(dim=-1).cpu().numpy()
            mpjpe[step * batch_size:step * batch_size + curr_batch_size] = error

            # Reconstuction_error
            r_error = reconstruction_error(pred_keypoints_3d.cpu().numpy(), gt_keypoints_3d.cpu().numpy(), reduction=None)
            recon_err[step * batch_size:step * batch_size + curr_batch_size] = r_error


        # If mask or part evaluation, render the mask and part images
        if eval_masks or eval_parts:
            mask, parts = renderer(pred_vertices, pred_camera)

        # Mask evaluation (for LSP)
        if eval_masks:
            center = batch['center'].cpu().numpy()
            scale = batch['scale'].cpu().numpy()
            # Dimensions of original image
            orig_shape = batch['orig_shape'].cpu().numpy()
            for i in range(curr_batch_size):
                # After rendering, convert imate back to original resolution
                pred_mask = uncrop(mask[i].cpu().numpy(), center[i], scale[i], orig_shape[i]) > 0
                # Load gt mask
                gt_mask = cv2.imread(os.path.join(annot_path, batch['maskname'][i]), 0) > 0
                # Evaluation consistent with the original UP-3D code
                accuracy += (gt_mask == pred_mask).sum()
                pixel_count += np.prod(np.array(gt_mask.shape))
                for c in range(2):
                    cgt = gt_mask == c
                    cpred = pred_mask == c
                    tp[c] += (cgt & cpred).sum()
                    fp[c] +=  (~cgt & cpred).sum()
                    fn[c] +=  (cgt & ~cpred).sum()
                f1 = 2 * tp / (2 * tp + fp + fn)

        # Part evaluation (for LSP)
        if eval_parts:
            center = batch['center'].cpu().numpy()
            scale = batch['scale'].cpu().numpy()
            orig_shape = batch['orig_shape'].cpu().numpy()
            for i in range(curr_batch_size):
                pred_parts = uncrop(parts[i].cpu().numpy().astype(np.uint8), center[i], scale[i], orig_shape[i])
                # Load gt part segmentation
                gt_parts = cv2.imread(os.path.join(annot_path, batch['partname'][i]), 0)
                # Evaluation consistent with the original UP-3D code
                # 6 parts + background
                for c in range(7):
                   cgt = gt_parts == c
                   cpred = pred_parts == c
                   cpred[gt_parts == 255] = 0
                   parts_tp[c] += (cgt & cpred).sum()
                   parts_fp[c] +=  (~cgt & cpred).sum()
                   parts_fn[c] +=  (cgt & ~cpred).sum()
                gt_parts[gt_parts == 255] = 0
                pred_parts[pred_parts == 255] = 0
                parts_f1 = 2 * parts_tp / (2 * parts_tp + parts_fp + parts_fn)
                parts_accuracy += (gt_parts == pred_parts).sum()
                parts_pixel_count += np.prod(np.array(gt_parts.shape))

        # Print intermediate results during evaluation
        if step % log_freq == log_freq - 1:
            if eval_pose:
                print('MPJPE: ' + str(1000 * mpjpe[:step * batch_size].mean()))
                print('Reconstruction Error: ' + str(1000 * recon_err[:step * batch_size].mean()))
                print()
            if eval_masks:
                print('Accuracy: ', accuracy / pixel_count)
                print('F1: ', f1.mean())
                print()
            if eval_parts:
                print('Parts Accuracy: ', parts_accuracy / parts_pixel_count)
                print('Parts F1 (BG): ', parts_f1[[0,1,2,3,4,5,6]].mean())
                print()
            if eval_orientation:
                print('Orientation error: ' + str(moe[:step * batch_size].mean()))

    # Save reconstructions to a file for further processing
    if save_results:
        np.savez(result_file, pred_joints=pred_joints, pose=smpl_pose, betas=smpl_betas, camera=smpl_camera)
    # Print final results during evaluation
    print('*** Final Results ***')
    print()
    if eval_pose:
        print('MPJPE: ' + str(1000 * mpjpe.mean()))
        print('Reconstruction Error: ' + str(1000 * recon_err.mean()))
        print()
    if eval_masks:
        print('Accuracy: ', accuracy / pixel_count)
        print('F1: ', f1.mean())
        print()
    if eval_parts:
        print('Parts Accuracy: ', parts_accuracy / parts_pixel_count)
        print('Parts F1 (BG): ', parts_f1[[0,1,2,3,4,5,6]].mean())
        print()
    if eval_orientation:
        print('Orientation Error: ' + str(moe.mean()))

In [ ]:
if __name__ == '__main__':
    
    # python3 eval.py --checkpoint=data/model_checkpoint.pt --dataset=h36m-p1 --log_freq=20 // example code
    args = parser.parse_args(['--checkpoint=data/model_checkpoint.pt','--dataset=3dpw', '--log_freq=20'])
    # Here we inserted our own arguments list
    
    model = hmr(config.SMPL_MEAN_PARAMS)
    checkpoint = torch.load(args.checkpoint)
    model.load_state_dict(checkpoint['model'], strict=False)
    model.eval()

    # Setup evaluation dataset
    dataset = BaseDataset(None, args.dataset, is_train=False)
    # Run evaluation
    run_evaluation(model, args.dataset, dataset, args.result_file,
                   batch_size=args.batch_size,
                   shuffle=args.shuffle,
                   log_freq=args.log_freq)

In [ ]:
# Our first sanity check should be whether ground truth compared to ground truth yields zero error

import torch 
import numpy as np
from scipy.spatial.transform import Rotation as R

import pickle as pkl
import os

img_path = 'examples/image_00502_crop.jpg'

pickle_path = 'data/3dpw/sequenceFiles/validation/courtyard_basketball_01.pkl'

frame = 502

# Load the .pkl sequence file containing the ground-truth information from 3dpw

seq = pkl.load(open(pickle_path,'rb'),encoding='latin-1') # opening the sequence file, latin-1 encoding for making it compatible with python3

gt_pose = torch.tensor(seq['poses'][0][frame]).unsqueeze(0)

# print(gt_pose)

pred_rotmat = torch.zeros(1, 24, 3, 3)

q = R.from_rotvec(gt_pose.reshape(1,24,-1)[0])

pred_rotmat = torch.tensor(R.as_dcm(q)).unsqueeze(0)

# print(pred_rotmat)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

curr_batch_size = 1

print("gt_pose", gt_pose.shape, gt_pose)

print("pred_rotmat", pred_rotmat.shape, pred_rotmat)

# Get ground truth orientation (already stored in gt_pose)
gt_rotvec = torch.zeros(curr_batch_size,24,3) # Have to have an array of this shape to input into the rotation object (from 32,72 to 32,24,3)
i = 0
for row in gt_pose:
    gt_rotvec[i] = torch.reshape(row,(24, -1))
    i+=1
print("gt_rotvec", gt_rotvec.shape, gt_rotvec)
    
gt_euler = np.zeros((curr_batch_size, 24, 3)) # Using numpy here because it works with the rotation library
i = 0
for row in gt_rotvec:
    r = R.from_rotvec(row)
    gt_euler[i] = R.as_euler(r, 'xyz', degrees=True)
    i+=1

print("gt_euler", gt_euler.shape, gt_euler)

# Get Euler representation of the predictions too:

pred_euler = np.zeros((curr_batch_size,24,3)) # Has to be a numpy array because it works with Rotation

# For each row in pred_rotmat convert it to a Rotation object and write it into a corresponding
# row in pred_euler as Euler angles

i=0
for row in pred_rotmat:
    r = R.from_dcm(row.cpu())
    pred_euler[i] = R.as_euler(r, 'xyz', degrees=True)
    i+=1

print("pred_euler", pred_euler.shape, pred_euler)
    
    
orientation_error_non_reduced = torch.sqrt((torch.from_numpy(gt_euler).to(device) -
                                torch.from_numpy(pred_euler).to(device))**2)
print(orientation_error_non_reduced)
    
orientation_error = torch.sqrt((torch.from_numpy(gt_euler).to(device) -
                                torch.from_numpy(pred_euler).to(device))**2).sum(dim=-1).mean(dim=-1)
# The reduction above is wrong. For a 90 degree error in one angle, it averages out 3.75 degrees, which
# is 90/24. The correct reduction would be a mean of 1.25 (90/72), because there are 72 angles (3 for each part)
# To remove the root, add [:,1:,:] to gt_euler and pred_euler above

orientation_error_new = torch.sqrt((torch.from_numpy(gt_euler).to(device) -
                                torch.from_numpy(pred_euler).to(device))**2).mean()
# This reduction is more accurate because it averages the error per part and then the error across parts
# It is equivalent to .mean(dim=-1).mean(dim=-1)

print("orientation_error")
print(orientation_error)
print()
print("orientation_error_new")
print(orientation_error_new)
print()

In [ ]:
def compare_orientation(gt, pred):
    # Taking as input two axis_angle representations
    
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    curr_batch_size = 1

    gt_pose = gt

    print("gt_pose", gt_pose.shape, gt_pose)

    q = R.from_rotvec(pred.reshape(1,24,-1)[0])

    pred_rotmat = torch.tensor(R.as_dcm(q)).unsqueeze(0)

    print("pred_rotmat", pred_rotmat.shape, pred_rotmat)

    # Get ground truth orientation (already stored in gt_pose)
    gt_rotvec = torch.zeros(curr_batch_size,24,3) # Have to have an array of this shape to input into the rotation object (from 32,72 to 32,24,3)
    i = 0
    for row in gt_pose:
        gt_rotvec[i] = torch.reshape(row,(24, -1))
        i+=1
    print("gt_rotvec", gt_rotvec.shape, gt_rotvec)

    gt_euler = np.zeros((curr_batch_size, 24, 3)) # Using numpy here because it works with the rotation library
    i = 0
    for row in gt_rotvec:
        r = R.from_rotvec(row)
        gt_euler[i] = R.as_euler(r, 'xyz', degrees=True)
        i+=1

    print("gt_euler", gt_euler.shape, gt_euler)

    # Get Euler representation of the predictions too:

    pred_euler = np.zeros((curr_batch_size,24,3)) # Has to be a numpy array because it works with Rotation

    # For each row in pred_rotmat convert it to a Rotation object and write it into a corresponding
    # row in pred_euler as Euler angles

    i=0
    for row in pred_rotmat:
        r = R.from_dcm(row.cpu())
        pred_euler[i] = R.as_euler(r, 'xyz', degrees=True)
        i+=1

    print("pred_euler", pred_euler.shape, pred_euler)


    orientation_error_non_reduced = torch.sqrt((torch.from_numpy(gt_euler).to(device) -
                                    torch.from_numpy(pred_euler).to(device))**2)
    print("error per part", orientation_error_non_reduced)

    orientation_error = torch.sqrt((torch.from_numpy(gt_euler).to(device) -
                                    torch.from_numpy(pred_euler).to(device))**2).sum(dim=-1).mean(dim=-1)
    # The reduction above is wrong. For a 90 degree error in one angle, it averages out 3.75 degrees, which
    # is 90/24. The correct reduction would be a mean of 1.25 (90/72), because there are 72 angles (3 for each part)
    # To remove the root, add [:,1:,:] to gt_euler and pred_euler above

    orientation_error_new = torch.sqrt((torch.from_numpy(gt_euler).to(device) -
                                    torch.from_numpy(pred_euler).to(device))**2).mean()
    # This reduction is more accurate because it averages the error per part and then the error across parts
    # It is equivalent to .mean(dim=-1).mean(dim=-1)

    print("orientation_error")
    print(orientation_error.item())
    print()
    print("orientation_error_new")
    print(orientation_error_new.item())
    print()

    #moe[step * batch_size:step * batch_size + curr_batch_size] = orientation_error.cpu()

In [24]:
import torch 
import numpy as np
from scipy.spatial.transform import Rotation as R

def compare_orientation_rotvec(gt, pred):
    # Taking as input two axis_angle representations
    
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    curr_batch_size = 1

    gt_pose = gt

    print("gt_pose", gt_pose.shape, gt_pose)
    print("pred_pose", pred.shape, pred)

    # Get ground truth orientation (already stored in gt_pose)
    gt_rotvec = torch.zeros((curr_batch_size,24,3), dtype=torch.double) # Have to have an array of this shape to input into the rotation object (from 32,72 to 32,24,3)
    i = 0
    for row in gt_pose:
        gt_rotvec[i] = torch.reshape(row,(24, -1))
        i+=1
    print("gt_rotvec", gt_rotvec.shape, gt_rotvec)
    
    # Get prediction as rotation vectors
    
    r = R.from_rotvec(pred.reshape(1,24,-1)[0])
    pred_rotvec = torch.tensor(R.as_rotvec(r)).unsqueeze(0)

    print("pred_rotvec", pred_rotvec.shape, pred_rotvec)

    orientation_error_non_reduced = np.degrees(torch.sqrt((gt_rotvec - pred_rotvec)**2))
    
    print("error per part", orientation_error_non_reduced)

    orientation_error = np.degrees(torch.sqrt((gt_rotvec - pred_rotvec)**2).sum(dim=-1).mean(dim=-1))
    # The reduction above is wrong. For a 90 degree error in one angle, it averages out 3.75 degrees, which
    # is 90/24. The correct reduction would be a mean of 1.25 (90/72), because there are 72 angles (3 for each part)
    # To remove the root, add [:,1:,:] to gt_euler and pred_euler above

    orientation_error_new = np.degrees(torch.sqrt((gt_rotvec - pred_rotvec)**2).mean())
    # This reduction is more accurate because it averages the error per part and then the error across parts
    # It is equivalent to .mean(dim=-1).mean(dim=-1)

    print("orientation_error")
    print(orientation_error.item())
    print()
    print("orientation_error_new")
    print(orientation_error_new.item())
    print()

In [25]:
a = torch.zeros(1,72)
b = torch.zeros(1,72)

b[0][0] = np.pi/4
b[0][1] = np.pi/4

compare_orientation_rotvec(a,b)

gt_pose torch.Size([1, 72]) tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
pred_pose torch.Size([1, 72]) tensor([[0.7854, 0.7854, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0

In [ ]:
m = [[(1/np.sqrt(2)),0,(1/np.sqrt(2))],
    [1/2, (1/np.sqrt(2)), -1/2],
    [-1/2, (1/np.sqrt(2)), 1/2]]

m

In [ ]:
# This is a rotation of 45 degrees around x multiplied on the
# left by a rotation of 45 degrees around y

m = [[(1/np.sqrt(2)),1/2,1/2],
    [0, (1/np.sqrt(2)), -(1/np.sqrt(2))],
    [-(1/np.sqrt(2)), 1/2, 1/2]]
m

In [ ]:
n = R.from_dcm(m)

In [ ]:
print(R.as_euler(n, 'xyz', degrees=True))

In [ ]:
o = R.from_euler('xyz', [45, 45, 0], degrees=True)

print(o.as_dcm())